In [14]:
"""
Проверка наличия данных из ведомости лесов в реестре УПП с отображением
"""
import pandas as pd
import openpyxl
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
import time

pd.options.mode.chained_assignment = None  # default='warn'

In [15]:
def convert_columns_to_str(df, number_columns):
    """
    Функция для конвертации указанных столбцов в строковый тип и очистки от пробельных символов в начале и конце
    """

    for column in number_columns:  # Перебираем список нужных колонок
        df.iloc[:, column] = df.iloc[:, column].astype(str)
        # Очищаем колонку от пробельных символов с начала и конца
        df.iloc[:, column] = df.iloc[:, column].apply(lambda x: x.strip())
        df.iloc[:, column] = df.iloc[:, column].apply(lambda x: x.replace(' ', ''))


In [16]:
path_to_end_folder = 'data/'
file_reestr_presense = 'data/Сравнение УПП с другими ведомостями на наличие участков или их отсутствие/2022-11-27_64_Реестр УПП с дополнительными колонками.xlsx'
file_statement_presense = 'data/Сравнение УПП с другими ведомостями на наличие участков или их отсутствие/Ведомость.xlsx'

In [17]:
# Считываем из файлов только те колонки по которым будет вестись сравнение
first_df = pd.read_excel(file_reestr_presense,
                         skiprows=8, usecols=[0,1,2,3,4], keep_default_na=False)
second_df = pd.read_excel(file_statement_presense,usecols=[0,1,2,3,4],keep_default_na=False)

In [18]:
# Приводим к строковому формату названия колонок первого датафрейма
first_df.columns = list(map(str,list(first_df.columns)))
# на всякий случай очищаем от пробельных символов
first_df.columns = list(map(lambda x:x.replace(" ",""),list(first_df.columns)))




In [19]:
# заменяем на цифры названия колонок во втором датафрейме
second_df.columns = ['1','2','3','4','5']

In [20]:
# Приводим датафреймы к строковому виду
first_df = first_df.astype(str)
second_df = second_df.astype(str)

In [21]:
# Очищаем от 
first_df.replace(r'^\s*$', 'Отсутствует', regex=True,inplace=True)
second_df.replace(r'^\s*$', 'Отсутствует', regex=True,inplace=True)

In [22]:
# так как мы заранее знаем сколько и какие колонки у нас есть то просто создаем список
params_columns = [0,1,2,3,4]
# Конвертируем нужные нам колонки в str
convert_columns_to_str(first_df, params_columns)
convert_columns_to_str(second_df, params_columns)

In [23]:
"""
Соответствие названий колонок используемым в программе номерам колонок
Лесничество -1
Участковое лесничество- 2
Урочище - 3
Номер лесного квартала -4
Номер лесотаксационного выдела -5

"""

'\nСоответствие названий колонок используемым в программе номерам колонок\nЛесничество -1\nУчастковое лесничество- 2\nУрочище - 3\nНомер лесного квартала -4\nНомер лесотаксационного выдела -5\n\n'

In [24]:
# Готовим 4 и 5 колонки чтобы они были разделеныпри склеивании
first_df['4'] = first_df['4'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
first_df['5'] = first_df['5'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел

# То же самое для второго датафрейма
second_df['4'] = second_df['4'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
second_df['5'] = second_df['5'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел


In [25]:
# Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
first_df['ID'] = first_df.iloc[:, params_columns].sum(axis=1)
second_df['ID'] = second_df.iloc[:, params_columns].sum(axis=1)

In [26]:
# Обрабатываем дубликаты


first_df.drop_duplicates(subset=['ID'], keep='last', inplace=True)  # Удаляем дубликаты из датафрейма

second_df.drop_duplicates(subset=['ID'], keep='last', inplace=True)  # Удаляем дубликаты из датафрейма






# Создаем документ
wb = openpyxl.Workbook()
# создаем листы
ren_sheet = wb['Sheet']
ren_sheet.title = 'Итог'

# Создаем датафрейм
itog_df = pd.merge(first_df, second_df, how='outer', left_on=['ID'], right_on=['ID'],
                   indicator=True)

# Отфильтровываем значения both,right
out_df = itog_df[(itog_df['_merge'] == 'both') | (itog_df['_merge'] == 'right_only')]

out_df.rename(columns={'_merge': 'Присутствие в реестре УПП','1_x':'Реестр Лесничество','2_x':'Реестр Уч.лесничество','3_x':'Реестр Урочище',
                       '4_x':'Реестр Квартал','5_x':'Реестр Выдел','1_y':'Ведомость Лесничество','2_y':'Ведомость Уч.Лесничество',
                       '3_y':'Ведомость Урочище','4_y':'Ведомость Квартал','5_y':'Ведомость Выдел'}, inplace=True)

out_df['Присутствие в реестре УПП'] = out_df['Присутствие в реестре УПП'].apply(
    lambda x: 'Имеется в реестре' if x == 'both' else 'Отсутствует в реестре')

# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
# Сохраняем отчет
# Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
wb = openpyxl.Workbook() # Создаем объект
# Записываем результаты
for row in dataframe_to_rows(out_df,index=False,header=True):
    wb['Sheet'].append(row)

# Форматирование итоговой таблицы
# Ширина колонок
wb['Sheet'].column_dimensions['A'].width = 15
wb['Sheet'].column_dimensions['B'].width = 20
wb['Sheet'].column_dimensions['C'].width = 10
wb['Sheet'].column_dimensions['G'].width = 20
wb['Sheet'].column_dimensions['H'].width = 20
wb['Sheet'].column_dimensions['F'].width = 50
wb['Sheet'].column_dimensions['L'].width = 30
# Перенос строк для заголовков
wb['Sheet']['D1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['E1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['F1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['G1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['H1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['I1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['J1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['K1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['H1'].alignment = Alignment(wrap_text=True)

wb.save(f'{path_to_end_folder}/Сравнение УПП с другими ведомостями на наличие участков или их отсутствие от  {current_time}.xlsx')